# PCA and k-means

## Setting up

In [ ]:
"""
Author      : Yi-Chieh Wu, Sriram Sankararaman
"""
import time

# numpy and scipy libraries
import numpy as np
from scipy import stats

# matplotlib libraries
import matplotlib.pyplot as plt
import collections
from collections import Counter, defaultdict

In [ ]:
import util
from util import *

## Point, Cluster and Set of Clusters classes

In [ ]:
######################################################################
# classes
######################################################################

class Point(object) :

    def __init__(self, name, label, attrs) :
        """
        A data point.

        Attributes
        --------------------
            name  -- string, name
            label -- string, label
            attrs -- numpy arrray of shape (d, ) where d is the number of features
        """

        self.name = name
        self.label = label
        self.attrs = attrs


    #============================================================
    # utilities
    #============================================================

    def distance(self, other) :
        """
        Return Euclidean distance of this point with other point.

        Parameters
        --------------------
            other -- Point, point to which we are measuring distance

        Returns
        --------------------
            dist  -- float, Euclidean distance
        """
        # Euclidean distance metric
        return np.linalg.norm(self.attrs-other.attrs)


    def __str__(self) :
        """
        Return string representation.
        """
        return "%s : (%s, %s)" % (self.name, str(self.attrs), self.label)

In [ ]:
class Cluster(object) :

    def __init__(self, points) :
        """
        A cluster (set of points).

        Attributes
        --------------------
            points -- list of Points, cluster elements
        """
        self.points = points


    def __str__(self) :
        """
        Return string representation.
        """
        s = ""
        for point in self.points :
            s += str(point)
        return s

    #============================================================
    # utilities
    #============================================================

    def purity(self) :
        """
        Compute cluster purity.

        Returns
        --------------------
            n           -- int, number of points in this cluster
            num_correct -- int, number of points in this cluster
                                with label equal to most common label in cluster
        """
        labels = []
        for p in self.points :
            labels.append(p.label)

        cluster_label, count = stats.mode(labels)
        return len(labels), np.float64(count)


    def centroid(self) :
        """
        Compute centroid of this cluster.

        Returns
        --------------------
            centroid -- Point, centroid of cluster
        """

        ### ========== TODO : START ========== ###
        # part 2b: implement
        # set the centroid label to any value (e.g. the most common label in this cluster)


        attrs_arr = np.array([p.attrs for p in self.points])
        centr_attr = np.mean(attrs_arr, axis=0)

        # Use Counter for efficient counting
        most_common_label = Counter(p.label for p in self.points).most_common(1)[0][0]

        centroid =  Point('centroid', most_common_label, centr_attr)
        return centroid
        ### ========== TODO : END ========== ###



    def medoid(self) :
        """
        Compute medoid of this cluster, that is, the point in this cluster
        that is closest to all other points in this cluster.

        Returns
        --------------------
            medoid -- Point, medoid of this cluster
        """

        ### ========== TODO : START ========== ###
        # part 2b: implement

        medoid = min(self.points, key=lambda p: sum(p.distance(k) for k in self.points))
        return medoid
        ### ========== TODO : END ========== ###


    def equivalent(self, other) :
        """
        Determine whether this cluster is equivalent to other cluster.
        Two clusters are equivalent if they contain the same set of points
        (not the same actual Point objects but the same geometric locations).

        Parameters
        --------------------
            other -- Cluster, cluster to which we are comparing this cluster

        Returns
        --------------------
            flag  -- bool, True if both clusters are equivalent or False otherwise
        """

        if len(self.points) != len(other.points) :
            return False

        matched = []
        for point1 in self.points :
            for point2 in other.points :
                if point1.distance(point2) == 0 and point2 not in matched :
                    matched.append(point2)
        return len(matched) == len(self.points)

In [ ]:
class ClusterSet(object):

    def __init__(self) :
        """
        A cluster set (set of clusters).

        Parameters
        --------------------
            members -- list of Clusters, clusters that make up this set
        """
        self.members = []


    #============================================================
    # utilities
    #============================================================

    def centroids(self) :
        """
        Return centroids of each cluster in this cluster set.

        Returns
        --------------------
            centroids -- list of Points, centroids of each cluster in this cluster set
        """

        ### ========== TODO : START ========== ###
        # part 2b: implement

        centroids = [member.centroid() for member in self.members]
        return centroids
        ### ========== TODO : END ========== ###


    def medoids(self) :
        """
        Return medoids of each cluster in this cluster set.

        Returns
        --------------------
            medoids -- list of Points, medoids of each cluster in this cluster set
        """

        ### ========== TODO : START ========== ###
        # part 2b: implement

        medoids = [member.medoid() for member in self.members]
        return medoids
        ### ========== TODO : END ========== ###


    def score(self) :
        """
        Compute average purity across clusters in this cluster set.

        Returns
        --------------------
            score -- float, average purity
        """

        total_correct = 0
        total = 0
        for c in self.members :
            n, n_correct = c.purity()
            total += n
            total_correct += n_correct
        return total_correct / float(total)


    def equivalent(self, other) :
        """
        Determine whether this cluster set is equivalent to other cluster set.
        Two cluster sets are equivalent if they contain the same set of clusters
        (as computed by Cluster.equivalent(...)).

        Parameters
        --------------------
            other -- ClusterSet, cluster set to which we are comparing this cluster set

        Returns
        --------------------
            flag  -- bool, True if both cluster sets are equivalent or False otherwise
        """

        if len(self.members) != len(other.members):
            return False

        matched = []
        for cluster1 in self.members :
            for cluster2 in other.members :
                if cluster1.equivalent(cluster2) and cluster2 not in matched:
                    matched.append(cluster2)
        return len(matched) == len(self.members)


    #============================================================
    # manipulation
    #============================================================

    def add(self, cluster):
        """
        Add cluster to this cluster set (only if it does not already exist).

        If the cluster is already in this cluster set, raise a ValueError.

        Parameters
        --------------------
            cluster -- Cluster, cluster to add
        """

        if cluster in self.members :
            raise ValueError

        self.members.append(cluster)

## k-means and k-medoids algorithms

In [ ]:
######################################################################
# k-means and k-medoids
######################################################################

def random_init(points, k) :
    """
    Randomly select k unique elements from points to be initial cluster centers.

    Parameters
    --------------------
        points         -- list of Points, dataset
        k              -- int, number of clusters

    Returns
    --------------------
        initial_points -- list of k Points, initial cluster centers
    """
    ### ========== TODO : START ========== ###
    # part 2c: implement (hint: use np.random.choice)
    return  np.random.choice(points, k, replace=False)


    ### ========== TODO : END ========== ###


def cheat_init(points) :
    """
    Initialize clusters by cheating!

    Details
    - Let k be number of unique labels in dataset.
    - Group points into k clusters based on label (i.e. class) information.
    - Return medoid of each cluster as initial centers.

    Parameters
    --------------------
        points         -- list of Points, dataset

    Returns
    --------------------
        initial_points -- list of k Points, initial cluster centers
    """
    ### ========== TODO : START ========== ###
    # part 2f: implement
    clusters = ClusterSet()
    grouped_points = defaultdict(list)

    # Group points by label in a single loop
    for p in points:
        grouped_points[p.label].append(p)

    # Create clusters from grouped points
    for label_points in grouped_points.values():
        clusters.add(Cluster(label_points))

    return clusters.medoids()

    ### ========== TODO : END ========== ###


def assign_clusters(center_points, points):
    closest = defaultdict(list)  # Maps centers to their assigned points

    for p in points:
        min_center = min(center_points, key=lambda center: p.distance(center))
        closest[min_center].append(p)

    return [Cluster(point_set) for point_set in closest.values()]


def kAverages(points, k, average, init='random', plot=False) :
    """
    Cluster points into k clusters using variations of k-means algorithm.

    Parameters
    --------------------
        points  -- list of Points, dataset
        k       -- int, number of clusters
        average -- method of ClusterSet
                   determines how to calculate average of points in cluster
                   allowable: ClusterSet.centroids, ClusterSet.medoids
        init    -- string, method of initialization
                   allowable:
                       'cheat'  -- use cheat_init to initialize clusters
                       'random' -- use random_init to initialize clusters
        plot    -- bool, True to plot clusters with corresponding averages
                         for each iteration of algorithm

    Returns
    --------------------
        k_clusters -- ClusterSet, k clusters
    """
    ### ========== TODO : START ========== ###
    # part 2c,2d: implement
    if init == "random":
        center_points = random_init(points, k)
    elif init == "cheat":
        center_points = cheat_init(points)
    else:
        raise ValueError("Invalid initialization method: choose 'random' or 'cheat'.")

    prev_ClusterSet = None
    curr_ClusterSet = ClusterSet()
    iteration = 1

    while True:
        # Assign points to the nearest cluster centers
        clusters = assign_clusters(center_points, points)
        curr_ClusterSet = ClusterSet()
        for cluster in clusters:
            curr_ClusterSet.add(cluster)

        # Check convergence
        if prev_ClusterSet and curr_ClusterSet.equivalent(prev_ClusterSet):
            return curr_ClusterSet

        prev_ClusterSet = curr_ClusterSet  # Update previous cluster set

        # Update centers based on selected average method
        if average == "centroids":
            center_points = curr_ClusterSet.centroids()
        elif average == "medoids":
            center_points = curr_ClusterSet.medoids()
        else:
            raise ValueError("Invalid average method: choose 'centroids' or 'medoids'.")

        # Plot if required
        if plot:
            plot_clusters(curr_ClusterSet,
                f"Iteration #{iteration} with {init} initialization ({average})",
                ClusterSet.centroids if average == "centroids" else ClusterSet.medoids)

        iteration += 1

    ### ========== TODO : END ========== ###


def kMeans(points, k, init='random', plot=False) :
    """
    Cluster points into k clusters using variations of k-means algorithm.

    Parameters
    --------------------
        points  -- list of Points, dataset
        k       -- int, number of clusters
        init    -- string, method of initialization
                   allowable:
                       'cheat'  -- use cheat_init to initialize clusters
                       'random' -- use random_init to initialize clusters
        plot    -- bool, True to plot clusters with corresponding averages
                         for each iteration of algorithm

    Returns
    --------------------
        k_clusters -- ClusterSet, k clusters
    """

    ### ========== TODO : START ========== ###
    # part 2c: implement
    # Hints:
    #   (1) On each iteration, keep track of the new cluster assignments
    #       in a separate data structure. Then use these assignments to create
    #       a new ClusterSet object and update the centroids.
    #   (2) Repeat until the clustering no longer changes.
    #   (3) To plot, use plot_clusters(...).

    return kAverages(points, k, "centroids", init, plot)
    ### ========== TODO : END ========== ###


def kMedoids(points, k, init='random', plot=False) :
    """
    Cluster points in k clusters using k-medoids clustering.
    See kMeans(...).
    """
    ### ========== TODO : START ========== ###
    # part 2e: implement
    return kAverages(points, k, "medoids", init, plot)
    ### ========== TODO : END ========== ###


## Utilities

In [ ]:
######################################################################
# helper functions
######################################################################

def build_face_image_points(X, y) :
    """
    Translate images to (labeled) points.

    Parameters
    --------------------
        X     -- numpy array of shape (n,d), features (each row is one image)
        y     -- numpy array of shape (n,), targets

    Returns
    --------------------
        point -- list of Points, dataset (one point for each image)
    """

    n,d = X.shape

    images = collections.defaultdict(list) # key = class, val = list of images with this class
    for i in range(n) :
        images[y[i]].append(X[i,:])

    points = []
    for face in images :
        count = 0
        for im in images[face] :
            points.append(Point(str(face) + '_' + str(count), face, im))
            count += 1

    return points


def plot_clusters(clusters, title, average) :
    """
    Plot clusters along with average points of each cluster.

    Parameters
    --------------------
        clusters -- ClusterSet, clusters to plot
        title    -- string, plot title
        average  -- method of ClusterSet
                    determines how to calculate average of points in cluster
                    allowable: ClusterSet.centroids, ClusterSet.medoids
    """

    plt.figure()
    np.random.seed(20)
    label = 0
    colors = {}
    centroids = average(clusters)
    for c in centroids :
        coord = c.attrs
        plt.plot(coord[0],coord[1], 'ok', markersize=12)
    for cluster in clusters.members :
        label += 1
        colors[label] = np.random.rand(3,)
        for point in cluster.points :
            coord = point.attrs
            plt.plot(coord[0], coord[1], 'o', color=colors[label])
    plt.title(title)
    plt.show()


def generate_points_2d(N, seed=1234) :
    """
    Generate toy dataset of 3 clusters each with N points.

    Parameters
    --------------------
        N      -- int, number of points to generate per cluster
        seed   -- random seed

    Returns
    --------------------
        points -- list of Points, dataset
    """
    np.random.seed(seed)

    mu = [[0,0.5], [1,1], [2,0.5]]
    sigma = [[0.1,0.1], [0.25,0.25], [0.15,0.15]]

    label = 0
    points = []
    for m,s in zip(mu, sigma) :
        label += 1
        for i in range(N) :
            x = random_sample_2d(m, s)
            points.append(Point(str(label)+'_'+str(i), label, x))

    return points

## Main function

In [ ]:
######################################################################
# main
######################################################################

def main() :
    ### ========== TODO : START ========== ###
    # part 1: explore LFW data set
    # part a

    ## Use get_lfw_data(...) to get the LFW dataset with labels,
    X, y = get_lfw_data()
    np.random.seed(42)  # Set the seed for reproducibility
    random_indices = np.random.choice(len(X), 5, replace=False)
    for i in random_indices:
        show_image(im=X[i])
    mean_face = np.mean(X, axis=0)
    show_image(mean_face)
    # part b
    ## how the top twelve eigenfaces using PCA
    U, mu = PCA(X)
    num_principal_component = 12
    plot_gallery([vec_to_image(U[:,i]) for i in range(num_principal_component)])
    # part c
    ## compare the original image with the reconstructions
    for l in [1, 10, 50, 100, 500, 1288]:
        Z, Ul = apply_PCA_from_Eig(X, U, l, mu)
        X_rec = reconstruct_from_PCA(Z, Ul, mu)
        print("\nl = ", l)
        plot_gallery([vec_to_image(X_rec[i]) for i in range(num_principal_component)])

    ### ========== TODO : END ========== ###



    ### ========== TODO : START ========== ###
    # part 2d-2f: cluster toy dataset
    points = generate_points_2d(20)
    clusterSet = kMeans(points, 3, plot=True)
    clusterSet = kMedoids(points, 3, plot=True)
    clusterSet = kMeans(points, 3, plot=True,init='cheat')
    clusterSet = kMedoids(points, 3, plot=True,init='cheat')
    ### ========== TODO : END ========== ###



    ### ========== TODO : START ========== ###
    # part 3a: cluster faces
    np.random.seed(1234) ## don't change the seed !!
    
    # Select images and convert to labeled points
    X1, y1 = util.limit_pics(X, y, [4, 6, 13, 16], 40)
    points = build_face_image_points(X1, y1)

    # Lists to store scores and times
    kMeans_scores, kMedoids_scores = [], []
    kMeans_times, kMedoids_times = [], []

    # Run clustering 10 times
    for _ in range(10):
        start = time.perf_counter()
        kMeanCluster = kMeans(points, 4)
        kMeans_times.append(time.perf_counter() - start)
        kMeans_scores.append(kMeanCluster.score())

        start = time.perf_counter()
        kMedoidsCluster = kMedoids(points, 4)
        kMedoids_times.append(time.perf_counter() - start)
        kMedoids_scores.append(kMedoidsCluster.score())

    # Convert lists to numpy arrays once
    kMeans_scores = np.array(kMeans_scores)
    kMedoids_scores = np.array(kMedoids_scores)
    kMeans_times = np.array(kMeans_times)
    kMedoids_times = np.array(kMedoids_times)

    # Compute statistics efficiently
    kMeans_avg, kMeans_min, kMeans_max = kMeans_scores.mean(), kMeans_scores.min(), kMeans_scores.max()
    kMeans_time_avg, kMeans_time_min, kMeans_time_max = kMeans_times.mean(), kMeans_times.min(), kMeans_times.max()

    kMedoids_avg, kMedoids_min, kMedoids_max = kMedoids_scores.mean(), kMedoids_scores.min(), kMedoids_scores.max()
    kMedoids_time_avg, kMedoids_time_min, kMedoids_time_max = kMedoids_times.mean(), kMedoids_times.min(), kMedoids_times.max()

    # Print results with min/max time included
    print(f"K-means: avg score: {kMeans_avg:.4f}, min: {kMeans_min:.4f}, max: {kMeans_max:.4f}, "
        f"avg time: {kMeans_time_avg:.4f}s, min time: {kMeans_time_min:.4f}s, max time: {kMeans_time_max:.4f}s")

    print(f"K-medoids: avg score: {kMedoids_avg:.4f}, min: {kMedoids_min:.4f}, max: {kMedoids_max:.4f}, "
        f"avg time: {kMedoids_time_avg:.4f}s, min time: {kMedoids_time_min:.4f}s, max time: {kMedoids_time_max:.4f}s")

    # part 3b: explore effect of lower-dimensional representations on clustering performance
    np.random.seed(1234) ## don't change the seed !!


    # Select images for classes 4 and 13
    X2, y2 = util.limit_pics(X, y, [4, 13], 40)

    # Compute PCA for the entire dataset
    U, mu = PCA(X)

    # Dictionaries to store scores
    kMeans_scores = {}
    kMedoids_scores = {}

    # Iterate over principal components with step size of 2
    for l in range(1, 42, 2):  
        # Project dataset into lower-dimensional space
        Z, Ul = apply_PCA_from_Eig(X2, U, l, mu)
        
        # Reconstruct dataset
        X2_rec = reconstruct_from_PCA(Z, Ul, mu)
        
        # Convert images into points
        points = build_face_image_points(X2_rec, y2)
        
        # Perform clustering using 'cheat' initialization
        kMeansCluster = kMeans(points, 2, init='cheat')
        kMedoidsCluster = kMedoids(points, 2, init='cheat')
        
        # Store clustering scores
        kMeans_scores[l] = kMeansCluster.score()
        kMedoids_scores[l] = kMedoidsCluster.score()

    # Create a single plot after the loop finishes
    plt.figure(figsize=(8, 5))
    plt.title("Effect of Lower-Dimensional Representations on Clustering Performance")
    plt.plot(kMeans_scores.keys(), kMeans_scores.values(), label='K-means', marker='o')
    plt.plot(kMedoids_scores.keys(), kMedoids_scores.values(), label='K-medoids', marker='s')

    # Label axes and show legend
    plt.xlabel("Number of Principal Components (l)")
    plt.ylabel("Clustering Score")
    plt.legend()
    plt.grid(True)
    plt.show()

    # part 3c: determine ``most discriminative'' and ``least discriminative'' pairs of images
    np.random.seed(1234) ## don't change the seed !!

    # Initialize max/min clustering performance pairs
    max_pair = (float('-inf'), None, None)  # Best clustering pair
    min_pair = (float('inf'), None, None)   # Worst clustering pair

    # Iterate over unique pairs (i, j) with j > i to avoid duplicate comparisons
    for i in range(19):
        for j in range(i + 1, 19):  
            X_ij, y_ij = util.limit_pics(X, y, [i, j], 40)
            points = build_face_image_points(X_ij, y_ij)

            # Apply k-Medoids clustering
            kMedoidsCluster = kMedoids(points, 2, init='cheat')
            score = kMedoidsCluster.score()

            # Update max_pair (best clustering performance)
            if score > max_pair[0]:
                max_pair = (score, i, j)

            # Update min_pair (worst clustering performance)
            if score < min_pair[0]:
                min_pair = (score, i, j)

    # Print results
    print(f"Best Clustering Pair: {max_pair}")
    print(f"Worst Clustering Pair: {min_pair}")

    # Visualize best and worst performing image pairs
    plot_representative_images(X, y, [max_pair[1], max_pair[2]], title="Best Performance")
    plot_representative_images(X, y, [min_pair[1], min_pair[2]], title="Worst Performance")
    ### ========== TODO : END ========== ###


if __name__ == "__main__" :
    main()